In [1]:
from pathlib import Path 

zhuanli_path_pdf = Path(r".\demo.pdf")
zhuanli_path_md = Path(r".\demo.md")

In [4]:
from __future__ import annotations
from pathlib import Path
import logging, os
from typing import List

from llama_index.core import Document, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import SentenceWindowNodeParser, SentenceSplitter
from llama_index.core.ingestion import run_transformations
from llama_index.core.schema import BaseNode, MetadataMode, NodeWithScore

from llama_index.readers.file import PyMuPDFReader
from llama_index.readers.file.markdown import MarkdownReader
import chromadb

# ---------- logging ----------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s.%(msecs)03d [%(levelname)-8s] %(name)25s - %(message)s",
    datefmt="%H:%M:%S",
)
log = logging.getLogger("patent_parser_min")

# ---------- paths ----------
PERSIST_DIR = Path("../temp").resolve()
PERSIST_DIR.mkdir(parents=True, exist_ok=True)
CHROMA_DIR = PERSIST_DIR / "chroma_db"

# ---------- components ----------
def build_embedding():
    return HuggingFaceEmbedding(model_name="Qwen/Qwen3-Embedding-0.6B")

def build_vector_store():
    client = chromadb.PersistentClient(path=str(CHROMA_DIR))
    collection = client.get_or_create_collection("patents")
    return ChromaVectorStore(chroma_collection=collection)

def read_file_to_documents(path: Path) -> List[Document]:
    ext = path.suffix.lower()
    if ext == ".pdf":
        return PyMuPDFReader().load_data(str(path))
    if ext == ".md":
        return MarkdownReader().load_data(str(path))
    # 兜底：按纯文本
    text = path.read_text(encoding="utf-8", errors="ignore")
    return [Document(text=text, metadata={"file_name": path.name})]

def show_nodes(nodes: List[BaseNode], max_n: int = 5):
    print("\n=== Sample BaseNodes ===")
    for i, n in enumerate(nodes[:max_n], 1):
        preview = n.get_content()[:160].replace("\n", " ")
        print(f"[{i}] node_id={n.node_id}  ref_doc_id={n.ref_doc_id}")
        print("    text:", preview, "..." if len(n.get_content()) > 160 else "")
        print("    meta:", {k: n.metadata.get(k) for k in list(n.metadata)[:6]})
        # 邻接
        print("    prev:", getattr(n.prev_node, "node_id", None), "  next:", getattr(n.next_node, "node_id", None))

def main(input_path: str, query: str | None = None, use_window: bool = True):
    path = Path(input_path).resolve()
    assert path.exists(), f"File not found: {path}"

    # 1) 读取文档
    docs = read_file_to_documents(path)
    for d in docs:
        d.metadata.setdefault("file_name", path.name)
        d.metadata["doc_id"] = d.doc_id
        # 与 private-gpt 一致：不把这些元数据送进 embedding / LLM
        d.excluded_embed_metadata_keys = ["doc_id"]
        d.excluded_llm_metadata_keys = ["file_name", "doc_id", "page_label"]

    # 2) 组件
    embed = build_embedding()
    vs = build_vector_store()
    storage = StorageContext.from_defaults(vector_store=vs)

    # 3) 选择分块器
    if use_window:
        # 与 private-gpt 默认一致：句子 + window（后续对话可用 MetadataReplacementPostProcessor("window")）
        parser = SentenceWindowNodeParser.from_defaults()  # window_size 默认 3
    else:
        # 定长切片示例
        parser = SentenceSplitter.from_defaults(chunk_size=600, chunk_overlap=200)

    # 4) 先直接跑 transformations 看看“裸的 Node”
    nodes = list(run_transformations(docs, [parser, embed], show_progress=True))
    show_nodes(nodes, max_n=5)

    # 5) 写索引 + 向量库（并持久化）
    index = VectorStoreIndex.from_documents(
        docs,
        storage_context=storage,
        embed_model=embed,
        transformations=[parser],   # 也可只给 parser，embedding 由 embed_model 负责
        show_progress=True,
    )
    index.storage_context.persist(persist_dir=str(PERSIST_DIR))
    print(f"\n✅ Persisted to: {PERSIST_DIR}")

    # 6) 即时检索（看 NodeWithScore）
    if query:
        retriever = index.as_retriever(similarity_top_k=5)
        results: List[NodeWithScore] = retriever.retrieve(query)
        print("\n=== Top matches ===")
        for r in results:
            txt = r.node.get_content()[:120].replace("\n", " ")
            print(f"score={round(r.score or 0, 3)}  node_id={r.node.node_id}  -> {txt} ...")

    # 7) 模拟“重新加载后”查看 DocStore 里的节点
    storage2 = StorageContext.from_defaults(vector_store=vs)
    _ = load_index_from_storage(storage2, persist_dir=str(PERSIST_DIR))
    ref_infos = storage2.docstore.get_all_ref_doc_info() or {}
    print(f"\n=== DocStore has {len(ref_infos)} ref docs ===")
    for doc_id, info in list(ref_infos.items())[:3]:
        print("ref_doc:", doc_id, "  meta keys:", list((info.metadata or {}).keys()))
        # 如果你的 LlamaIndex 版本携带 node_ids，可以尝试：
        node_ids = getattr(info, "node_ids", None)
        if node_ids:
            some_nodes = storage2.docstore.get_nodes(node_ids[:3])
            print("  sample nodes:", [n.node_id for n in some_nodes])

if __name__ == "__main__":
    
    inp = r"./demo.md"
    q = "解析一下专利的摘要"
    main(inp, q)
    

2025-09-01 17:09:35,581 - INFO - Load pretrained SentenceTransformer: Qwen/Qwen3-Embedding-0.6B


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

Generating embeddings:   0%|          | 0/31 [00:00<?, ?it/s]


✅ Persisted to: D:\ddesktop\agentdemos\codespace\raga\scripts\local_data


2025-09-01 17:13:56,740 - INFO - Loading all indices.



=== Top matches ===
score=0.293  node_id=469e6744-4250-477c-8ab6-1af0348aebe9  ->   (54)实用新型名称 一种结构紧凑的回转动力单元以及应用其的机器人 ...
score=0.29  node_id=499b6f68-9ef1-4768-88ff-f8f3707af396  ->   (12)实用新型专利 (21)申请号202021894937.5 (22)申请日2020.09.02 (73)专利权人杭州宇树科技有限公司地址310053浙江省杭州市滨江区西兴街道东流路88号1幢306室 (72)发明人王兴兴 (74) ...
score=0.287  node_id=66b2dc03-0771-4778-a5a4-9f5e327c296b  ->   (19)中华人民共和国国家知识产权局  ...
score=0.258  node_id=7dbc70f1-aaa2-46dd-a855-a9b5c76ec699  ->   技术领域 [0001] 本实用新型涉及一种结构紧凑的回转动力单元以及应用其的机器人，属于动力单元以及机器人技术领域。 ...
score=0.254  node_id=352c6335-df88-4bb1-aa89-807dbf53bec1  ->   一种结构紧凑的回转动力单元以及应用其的机器人  ...


ValueError: No index in storage context, check if you specified the right persist_dir.

In [ ]:
# llama-index 解析 专利pdf、专利md（MinerU解析得到的md）

from pydantic import BaseModel, Field
from typing import Literal, Any, List, Tuple, Union, Optional 
import abc,  itertools, multiprocessing, multiprocessing.pool, threading
from injector import inject, Injector, singleton 
from queue import Queue


from llama_index.core.data_structs import IndexDict
from llama_index.core.embeddings.utils import EmbedType
from llama_index.core.indices import VectorStoreIndex, load_index_from_storage
from llama_index.core.indices.base import BaseIndex
from llama_index.core.ingestion import run_transformations
from llama_index.core.schema import BaseNode, Document, TransformComponent
from llama_index.core.storage import StorageContext
from llama_index.core.readers import StringIterableReader
from llama_index.core.readers.base import BaseReader
from llama_index.core.readers.json import JSONReader




import logging
# Set to 'DEBUG' to have extensive logging turned on, even for libraries
ROOT_LOG_LEVEL = "INFO"
PRETTY_LOG_FORMAT = (
    "%(asctime)s.%(msecs)03d [%(levelname)-8s] %(name)+25s - %(message)s"
)
logging.basicConfig(level=ROOT_LOG_LEVEL, format=PRETTY_LOG_FORMAT, datefmt="%H:%M:%S")
logging.captureWarnings(True)
logger = logging.getLogger(__name__)

# setting
local_data_path = zhuanli_path_pdf




# file loader    【"zhuanli.pdf", "zhuanli.md"】 "image.jpg"
def _try_loading_included_file_formats() -> dict[str, type[BaseReader]]:
    try:
        from llama_index.readers.file.docs import PDFReader
        from llama_index.readers.file.markdown import MarkdownReader
        from llama_index.readers.file.image import ImageReader  # type: ignore
    except ImportError as e: raise ImportError("import error")
    
    default_file_reader_cls: dict[str, type[BaseReader]] = {
        ".pdf": PDFReader,
        ".md": MarkdownReader,
        ".jpg": ImageReader,
        ".png": ImageReader,
        ".jpeg": ImageReader,
    }
    return default_file_reader_cls
file_reader_cls = _try_loading_included_file_formats()
file_reader_cls.update({".json": JSONReader})
class IngestionHelper:
    """Helper class to transform a file into a list of documents.

    This class should be used to transform a file into a list of documents.
    These methods are thread-safe (and multiprocessing-safe).
    """

    @staticmethod
    def transform_file_into_documents(
        file_name: str, file_data: Path
    ) -> list[Document]:
        documents = IngestionHelper._load_file_to_documents(file_name, file_data)
        for document in documents:
            document.metadata["file_name"] = file_name
        IngestionHelper._exclude_metadata(documents)
        return documents

    @staticmethod
    def _load_file_to_documents(file_name: str, file_data: Path) -> list[Document]:
        logger.debug("Transforming file_name=%s into documents", file_name)
        extension = Path(file_name).suffix
        reader_cls = file_reader_cls.get(extension)
        if reader_cls is None:
            logger.debug(
                "No reader found for extension=%s, using default string reader",
                extension,
            )
            # Read as a plain text
            string_reader = StringIterableReader()
            return string_reader.load_data([file_data.read_text()])

        logger.debug("Specific reader found for extension=%s", extension)
        documents = reader_cls().load_data(file_data)

        # Sanitize NUL bytes in text which can't be stored in Postgres
        for i in range(len(documents)):
            documents[i].text = documents[i].text.replace("\u0000", "")

        return documents

    @staticmethod
    def _exclude_metadata(documents: list[Document]) -> None:
        logger.debug("Excluding metadata from count=%s documents", len(documents))
        for document in documents:
            document.metadata["doc_id"] = document.doc_id
            # We don't want the Embeddings search to receive this metadata
            document.excluded_embed_metadata_keys = ["doc_id"]
            # We don't want the LLM to receive these metadata in the context
            document.excluded_llm_metadata_keys = ["file_name", "doc_id", "page_label"]
    




class IngestedDoc(BaseModel):
    object: Literal["ingest.document"]
    doc_id: str = Field(examples=["c202d5e6-7b69-4869-81cc-dd574ee8ee11"])
    doc_metadata: dict[str, Any] | None = Field(
        examples=[
            {
                "page_label": "2",
                "file_name": "Sales Report Q3 2023.pdf",
            }
        ]
    )

    @staticmethod
    def curate_metadata(metadata: dict[str, Any]) -> dict[str, Any]:
        """Remove unwanted metadata keys."""
        for key in ["doc_id", "window", "original_text"]:
            metadata.pop(key, None)
        return metadata

    @staticmethod
    def from_document(document: Document) -> "IngestedDoc":
        return IngestedDoc(
            object="ingest.document",
            doc_id=document.doc_id,
            doc_metadata=IngestedDoc.curate_metadata(document.metadata),
        )

class BaseIngestComponent(abc.ABC):
    def __init__(
        self,
        storage_context: StorageContext,
        embed_model: EmbedType,
        transformations: list[TransformComponent],
        *args: Any,
        **kwargs: Any,
    ) -> None:
        logger.debug("Initializing base ingest component type=%s", type(self).__name__)
        self.storage_context = storage_context
        self.embed_model = embed_model
        self.transformations = transformations

    @abc.abstractmethod
    def ingest(self, file_name: str, file_data: Path) -> list[Document]:
        pass

    @abc.abstractmethod
    def bulk_ingest(self, files: list[tuple[str, Path]]) -> list[Document]:
        pass

    @abc.abstractmethod
    def delete(self, doc_id: str) -> None:
        pass

class BaseIngestComponentWithIndex(BaseIngestComponent, abc.ABC):
    def __init__(
        self,
        storage_context: StorageContext,
        embed_model: EmbedType,
        transformations: list[TransformComponent],
        *args: Any,
        **kwargs: Any,
    ) -> None:
        super().__init__(storage_context, embed_model, transformations, *args, **kwargs)

        self.show_progress = True
        self._index_thread_lock = (
            threading.Lock()
        )  # Thread lock! Not Multiprocessing lock
        self._index = self._initialize_index()

    def _initialize_index(self) -> BaseIndex[IndexDict]:
        """Initialize the index from the storage context."""
        try:
            # Load the index with store_nodes_override=True to be able to delete them
            index = load_index_from_storage(
                storage_context=self.storage_context,
                store_nodes_override=True,  # Force store nodes in index and document stores
                show_progress=self.show_progress,
                embed_model=self.embed_model,
                transformations=self.transformations,
            )
        except ValueError:
            # There are no index in the storage context, creating a new one
            logger.info("Creating a new vector store index")
            index = VectorStoreIndex.from_documents(
                [],
                storage_context=self.storage_context,
                store_nodes_override=True,  # Force store nodes in index and document stores
                show_progress=self.show_progress,
                embed_model=self.embed_model,
                transformations=self.transformations,
            )
            index.storage_context.persist(persist_dir=local_data_path)
        return index

    def _save_index(self) -> None:
        self._index.storage_context.persist(persist_dir=local_data_path)

    def delete(self, doc_id: str) -> None:
        with self._index_thread_lock:
            # Delete the document from the index
            self._index.delete_ref_doc(doc_id, delete_from_docstore=True)

            # Save the index
            self._save_index()

class BatchIngestComponent(BaseIngestComponentWithIndex):
    """Parallelize the file reading and parsing on multiple CPU core.

    This also makes the embeddings to be computed in batches (on GPU or CPU).
    """

    def __init__(
        self,
        storage_context: StorageContext,
        embed_model: EmbedType,
        transformations: list[TransformComponent],
        count_workers: int,
        *args: Any,
        **kwargs: Any,
    ) -> None:
        super().__init__(storage_context, embed_model, transformations, *args, **kwargs)
        # Make an efficient use of the CPU and GPU, the embedding
        # must be in the transformations
        assert (
            len(self.transformations) >= 2
        ), "Embeddings must be in the transformations"
        assert count_workers > 0, "count_workers must be > 0"
        self.count_workers = count_workers

        self._file_to_documents_work_pool = multiprocessing.Pool(
            processes=self.count_workers
        )

    def ingest(self, file_name: str, file_data: Path) -> list[Document]:
        logger.info("Ingesting file_name=%s", file_name)
        documents = IngestionHelper.transform_file_into_documents(file_name, file_data)
        logger.info(
            "Transformed file=%s into count=%s documents", file_name, len(documents)
        )
        logger.debug("Saving the documents in the index and doc store")
        return self._save_docs(documents)

    def bulk_ingest(self, files: list[tuple[str, Path]]) -> list[Document]:
        documents = list(
            itertools.chain.from_iterable(
                self._file_to_documents_work_pool.starmap(
                    IngestionHelper.transform_file_into_documents, files
                )
            )
        )
        logger.info(
            "Transformed count=%s files into count=%s documents",
            len(files),
            len(documents),
        )
        return self._save_docs(documents)

    def _save_docs(self, documents: list[Document]) -> list[Document]:
        logger.debug("Transforming count=%s documents into nodes", len(documents))
        nodes = run_transformations(
            documents,  # type: ignore[arg-type]
            self.transformations,
            show_progress=self.show_progress,
        )
        # Locking the index to avoid concurrent writes
        with self._index_thread_lock:
            logger.info("Inserting count=%s nodes in the index", len(nodes))
            self._index.insert_nodes(nodes, show_progress=True)
            for document in documents:
                self._index.docstore.set_document_hash(
                    document.get_doc_id(), document.hash
                )
            logger.debug("Persisting the index and nodes")
            # persist the index and nodes
            self._save_index()
            logger.debug("Persisted the index and nodes")
        return documents



def get_ingestion_component(
    storage_context: StorageContext,
    embed_model: EmbedType,
    transformations: list[TransformComponent],
    settings: Settings,
) -> BaseIngestComponent:
    """Get the ingestion component for the given configuration."""
    ingest_mode = settings.embedding.ingest_mode
    if ingest_mode == "batch":
        return BatchIngestComponent(
            storage_context=storage_context,
            embed_model=embed_model,
            transformations=transformations,
            count_workers=settings.embedding.count_workers,
        )

    elif ingest_mode == "pipeline":
        return PipelineIngestComponent(
            storage_context=storage_context,
            embed_model=embed_model,
            transformations=transformations,
            count_workers=settings.embedding.count_workers,
        )
    else:
        return SimpleIngestComponent(
            storage_context=storage_context,
            embed_model=embed_model,
            transformations=transformations,
        )
    



@singleton
class IngestService:
    @inject
    def __init__(
        self,
        llm_component: LLMComponent,
        vector_store_component: VectorStoreComponent,
        embedding_component: EmbeddingComponent,
        node_store_component: NodeStoreComponent,
    ) -> None:
        self.llm_service = llm_component
        self.storage_context = StorageContext.from_defaults(
            vector_store=vector_store_component.vector_store,
            docstore=node_store_component.doc_store,
            index_store=node_store_component.index_store,
        )
        node_parser = SentenceWindowNodeParser.from_defaults()

        self.ingest_component = get_ingestion_component(
            self.storage_context,
            embed_model=embedding_component.embedding_model,
            transformations=[node_parser, embedding_component.embedding_model],
            settings=settings(),
        )

    def _ingest_data(self, file_name: str, file_data: AnyStr) -> list[IngestedDoc]:
        logger.debug("Got file data of size=%s to ingest", len(file_data))
        # llama-index mainly supports reading from files, so
        # we have to create a tmp file to read for it to work
        # delete=False to avoid a Windows 11 permission error.
        with tempfile.NamedTemporaryFile(delete=False) as tmp:
            try:
                path_to_tmp = Path(tmp.name)
                if isinstance(file_data, bytes):
                    path_to_tmp.write_bytes(file_data)
                else:
                    path_to_tmp.write_text(str(file_data))
                return self.ingest_file(file_name, path_to_tmp)
            finally:
                tmp.close()
                path_to_tmp.unlink()

    def ingest_file(self, file_name: str, file_data: Path) -> list[IngestedDoc]:
        logger.info("Ingesting file_name=%s", file_name)
        documents = self.ingest_component.ingest(file_name, file_data)
        logger.info("Finished ingestion file_name=%s", file_name)
        return [IngestedDoc.from_document(document) for document in documents]

    def ingest_text(self, file_name: str, text: str) -> list[IngestedDoc]:
        logger.debug("Ingesting text data with file_name=%s", file_name)
        return self._ingest_data(file_name, text)

    def ingest_bin_data(
        self, file_name: str, raw_file_data: BinaryIO
    ) -> list[IngestedDoc]:
        logger.debug("Ingesting binary data with file_name=%s", file_name)
        file_data = raw_file_data.read()
        return self._ingest_data(file_name, file_data)

    def bulk_ingest(self, files: list[tuple[str, Path]]) -> list[IngestedDoc]:
        logger.info("Ingesting file_names=%s", [f[0] for f in files])
        documents = self.ingest_component.bulk_ingest(files)
        logger.info("Finished ingestion file_name=%s", [f[0] for f in files])
        return [IngestedDoc.from_document(document) for document in documents]

    def list_ingested(self) -> list[IngestedDoc]:
        ingested_docs: list[IngestedDoc] = []
        try:
            docstore = self.storage_context.docstore
            ref_docs: dict[str, RefDocInfo] | None = docstore.get_all_ref_doc_info()

            if not ref_docs:
                return ingested_docs

            for doc_id, ref_doc_info in ref_docs.items():
                doc_metadata = None
                if ref_doc_info is not None and ref_doc_info.metadata is not None:
                    doc_metadata = IngestedDoc.curate_metadata(ref_doc_info.metadata)
                ingested_docs.append(
                    IngestedDoc(
                        object="ingest.document",
                        doc_id=doc_id,
                        doc_metadata=doc_metadata,
                    )
                )
        except ValueError:
            logger.warning("Got an exception when getting list of docs", exc_info=True)
            pass
        logger.debug("Found count=%s ingested documents", len(ingested_docs))
        return ingested_docs

    def delete(self, doc_id: str) -> None:
        """Delete an ingested document.

        :raises ValueError: if the document does not exist
        """
        logger.info(
            "Deleting the ingested document=%s in the doc and index store", doc_id
        )
        self.ingest_component.delete(doc_id)
 



